In [1]:
from exjobb_data import *

Using TensorFlow backend.


exjobb data prepare loading...   
data loaded ....


In [2]:
with open('CarBinarizer', 'rb') as f:
    cb = pickle.load(f)
with open('VolvoStats', 'rb') as f:
    stats = pickle.load(f)

In [3]:
import types
import tempfile
import keras.models

def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            keras.models.save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = { 'model_str': model_str }
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = keras.models.load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = keras.models.Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__

In [4]:
train,test,dev,train_ct,train_eng,train_sv,train_gb,train_col,train_uph=set_data(use_new=True)

using new data...


In [5]:
test.head(0)

,antal_inpendlare,antal_utpendlare,förvärvs-arbetande,ej_förvärvs-arbetande,äganderätt/småhus,bostadsrätt,hyresrätt,övriga_inkl._uppgift_saknas,förgymnasial,gymnasial,eftergymnasial_mindre_än_3_år,eftergymnasial_3_år_eller_längre_inkl._forskarutbildning,0-6_år,7-15_år,16-19_år,20-24_år,25-44_år,45-64_år,65-w_år,låg_inkomst,medellåg_inkomst,medelhög_inkomst,hög_inkomst,medianinkomst,sammanboende_med_barn,sammanboende_utan_barn,ensamstående_med_barn,ensamstående_utan_barn,övriga_hushåll,låg_köpkraft,medellåg_köpkraft,medelhög_köpkraft,hög_köpkraft,median_köpkraft,"jordbruk,_skogsbruk,_jakt_och_fiske",tillverkning_och_utvinning,energi_och_miljöverksamhet,byggverksamhet,handel,transport_och_magasinering,hotell-_och_restaurangverksamhet,information_och_kommunikation,finans-_och_försäkringsverksamhet,fastighetsverksamhet,företagstjänster,offentlig_förvaltning_och_försvar,utbildning,"vård_och_omsorg,_sociala_tjänster",kulturella_och_personliga_tjänster_m.m.,okänd_verksamhet,0_barn,1_barn,2_barn,3+_barn,TYPECODE_onehot,ENGINECODE_onehot,SALESVERSIONCODE_onehot,GEARBOXCODE_onehot,COLOUR_onehot,UPHOLSTERY_onehot,OPT_CODES_onehot,TYPECODE_translated,ENGINECODE_translated,SALESVERSIONCODE_translated,GEARBOXCODE_translated,COLOUR_translated,UPHOLSTERY_translated,OPT_CODES_translated


In [6]:
model_dict = {}
for idx in range(len(pno12_solver.pno12s)):
    _,ct,eng,sv,gb = re.findall(r'(..PNO12..)(...)_(..)_(..)_(.)',pno12_solver.pno12s[idx][0])[0]
    c2i = pno12_solver.solver_data['css'][ct].c2i
    
    colors=list(map(lambda x: x[-5:],filter(lambda x: 'COL' in x,c2i.keys())))
    uphs=list(map(lambda x: x[-6:],filter(lambda x: 'UPH' in x,c2i.keys())))
    opts=list(map(lambda x: x[-6:],filter(lambda x: 'OPT' in x,c2i.keys())))
    
    if not ct in model_dict:
        model_dict[ct] = {
            'ENGINE':set([eng]),
            'SALES_VERSION':set([sv]),
            'GEARBOX': set([gb]),
            'OPT':set(opts),
            'COL':set(colors),
            'UPH':set(uphs)
        }
    else:
        model_dict[ct]['ENGINE'].add(eng)
        model_dict[ct]['SALES_VERSION'].add(sv)
        model_dict[ct]['GEARBOX'].add(gb)
        model_dict[ct]['OPT'].update(opts)
        model_dict[ct]['UPH'].update(uphs)
        model_dict[ct]['COL'].update(colors)        


In [9]:
class BNN:
    def __init__(self,structure,ctx_size,cb,should_build=True):
        self.cb = cb
        self.blue_print = structure
        self.structures = {}
        self.ctx_size = ctx_size
        self.ct_encoder = LabelBinarizer()
        self.ct_encoder.fit(list(self.blue_print.keys()))
        self.label_encoders = {}
        if should_build:
            self._build_model()
        self.scaler = MinMaxScaler()
        self.scaler_fitted = False
        
    def _build_model(self):
        ## Begin with predict ct:
        
        ctx_in = Input(shape=(self.ctx_size,))
        dense = Dense(1000, activation='relu')(ctx_in)
        dense = Dense(128, activation='relu')(dense)
        #lenblueprint = number of car_types
        ct_out = Dense(len(self.blue_print), activation='softmax')(dense)
        self.ct_predictor = Model(ctx_in,ct_out,name='ct_model')
        self.ct_predictor.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.001),metrics=['accuracy'])
        
        for ct in self.blue_print.keys():
            
            #label_encoders
            eng_enc = LabelBinarizer()
            gb_enc = LabelBinarizer()
            sv_enc = LabelBinarizer()
            col_enc = LabelBinarizer()
            uph_enc = LabelBinarizer()
            opt_enc = MultiLabelBinarizer()
            
            engines = self.blue_print[ct]['ENGINE']
            gearboxes = self.blue_print[ct]['GEARBOX']
            sales_versions = self.blue_print[ct]['SALES_VERSION']
            colors = self.blue_print[ct]['COL']
            uphs = self.blue_print[ct]['UPH']
            opts = self.blue_print[ct]['OPT']
            
            eng_enc.fit(list(engines))
            gb_enc.fit(list(gearboxes))
            sv_enc.fit(list(sales_versions))
            uph_enc.fit(list(uphs))
            col_enc.fit(list(colors))
            opt_enc.fit([list(opts)])
            
            self.label_encoders[ct]={
                'ENGINE':eng_enc,
                'GEARBOX':gb_enc,
                'SALES_VERSION':sv_enc,
                'UPH': uph_enc,
                'COL': col_enc,
                'OPT': opt_enc
            }
                      
            eng_predictor = self.create_model(ctx_in, self.get_layer_size(len(engines)), layers=[128,128], name=f'{ct}_eng_model')
            sv_predictor = self.create_model(ctx_in,self.get_layer_size(len(sales_versions)),layers=[128,128],name=f'{ct}_sv_model')
            eng_dependency = Input(shape=(self.get_layer_size(len(engines)),))
            gb_predictor = self.create_model([ctx_in, eng_dependency],self.get_layer_size(len(gearboxes)), layers=[128,128],name=f'{ct}_gearbox_model')
            col_predictor = self.create_model(ctx_in, self.get_layer_size(len(colors)), layers=[128,128], name=f'{ct}_col_model')
            uph_predictor = self.create_model(ctx_in, self.get_layer_size(len(uphs)), layers=[128,128], name=f'{ct}_uph_model')
            opt_predictor = self.create_model(ctx_in, self.get_layer_size(len(opts)), layers=[128,128], name=f'{ct}_opt_model', is_multilabel=True)
            
            self.structures[ct] = {
                'ENGINE': eng_predictor,
                'GEARBOX':gb_predictor,
                'SALES_VERSION': sv_predictor,
                'COL': col_predictor,
                'UPH':uph_predictor,
                'OPT':opt_predictor
            }  
    
    def get_layer_size(self,layer_size):
        return layer_size if layer_size > 2 else 1
    
    
    def create_model(self,inputs,output_size,layers,name,is_multilabel=False):
        """
            inputs=actual inputs, either one or a list
            output_size=number of units in last layer
            layers = [1,2,3,4],
            name = model name
        """
        
        first_layer = None
        if isinstance(inputs,list):
            first_layer = concatenate(inputs)
        else:
            first_layer = Dense(128,activation='relu')(inputs)

        wrapper_dense = Dense(layers[0],activation='relu')(first_layer)
        for layer_size in layers[1:]:
            wrapper_dense = Dense(layer_size, activation='relu')(Dropout(0.3)(wrapper_dense))
        
        out_layer_size = self.get_layer_size(output_size)
        out_activation = 'softmax' if out_layer_size > 1 else 'sigmoid'
        loss = 'categorical_crossentropy' if out_layer_size > 1 else 'binary_crossentropy'
        print(f'{name}, computed output size: {out_layer_size}, specified size: {output_size}')
        
        if is_multilabel:
            out_activation = 'sigmoid'
            loss = 'binary_crossentropy'
        
        out_layer = Dense(out_layer_size, activation=out_activation,name=f'last_out_{name}')(wrapper_dense)
        model = Model(inputs=inputs, outputs=out_layer, name=name)
        model.compile(loss=loss, optimizer=optimizers.Adam(lr=0.001),metrics=['accuracy'])
        #print(model.summary())
        return model
    

    def add_training_data(self,contexts,targets):
        
        self.train_on = {}
        self.validate_on=None
        seen = []
        result = targets.progress_apply(lambda row: (self.label_encoders[row['TYPECODE_translated']]['ENGINE'].transform([row['ENGINECODE_translated']]),\
                                                    self.label_encoders[row['TYPECODE_translated']]['GEARBOX'].transform([row['GEARBOXCODE_translated']]),\
                                                     self.label_encoders[row['TYPECODE_translated']]['SALES_VERSION'].transform([row['SALESVERSIONCODE_translated']]),\
                                                     self.label_encoders[row['TYPECODE_translated']]['COL'].transform([row['COLOUR_translated']]),\
                                                     self.label_encoders[row['TYPECODE_translated']]['UPH'].transform([row['UPHOLSTERY_translated']]),\
                                                     self.label_encoders[row['TYPECODE_translated']]['OPT'].transform([row['OPT_CODES_translated']])),axis=1)
        result = list(zip(*result))
        engs_oh = list(result[0])
        gbs_oh = list(result[1])
        sv_oh = list(result[2])
        col_oh = list(result[3])
        uph_oh = list(result[4])
        opt_oh = list(result[5])
        
        
        
   
        cts = targets['TYPECODE_translated']
        for ctx, ct, eng, gb, sv,col,uph,opt in tqdm_notebook(zip(contexts.values,cts.values,engs_oh,gbs_oh,sv_oh,col_oh,uph_oh,opt_oh)):
            if not ct in self.train_on:
                self.train_on[ct]={
                    'contexts':[],
                    'ENGINE':[],
                    'GEARBOX':[],
                    'SALES_VERSION':[],
                    'UPH':[],
                    'COL':[],
                    'OPT':[]
                    }
            if ct in seen:
                continue

            self.train_on[ct]['contexts']+=[ctx]
            self.train_on[ct]['ENGINE']+=[eng[0]]
            self.train_on[ct]['GEARBOX']+=[gb[0]]
            self.train_on[ct]['SALES_VERSION']+=[sv[0]]
            self.train_on[ct]['COL']+=[col[0]]
            self.train_on[ct]['UPH']+=[uph[0]]
            self.train_on[ct]['OPT']+=[opt[0]]         
            
    def add_validation_data(self,contexts,targets):
        self.validate_on = {}
        seen = []
        result = targets.progress_apply(lambda row: (self.label_encoders[row['TYPECODE_translated']]['ENGINE'].transform([row['ENGINECODE_translated']]),\
                                                    self.label_encoders[row['TYPECODE_translated']]['GEARBOX'].transform([row['GEARBOXCODE_translated']]),\
                                                     self.label_encoders[row['TYPECODE_translated']]['SALES_VERSION'].transform([row['SALESVERSIONCODE_translated']]),\
                                                     self.label_encoders[row['TYPECODE_translated']]['COL'].transform([row['COLOUR_translated']]),\
                                                     self.label_encoders[row['TYPECODE_translated']]['UPH'].transform([row['UPHOLSTERY_translated']]),\
                                                     self.label_encoders[row['TYPECODE_translated']]['OPT'].transform([row['OPT_CODES_translated']])),axis=1)
        result = list(zip(*result))
        engs_oh = list(result[0])
        gbs_oh = list(result[1])
        sv_oh = list(result[2])
        col_oh = list(result[3])
        uph_oh = list(result[4])
        opt_oh = list(result[5])
        
        cts = targets['TYPECODE_translated']
        for ctx, ct, eng, gb, sv,col,uph,opt in tqdm_notebook(zip(contexts.values,cts.values,engs_oh,gbs_oh,sv_oh,col_oh,uph_oh,opt_oh)):
            if not ct in self.validate_on:
                self.validate_on[ct]={
                    'contexts':[],
                    'ENGINE':[],
                    'GEARBOX':[],
                    'SALES_VERSION':[],
                    'UPH':[],
                    'COL':[],
                    'OPT':[]
                    }
            if ct in seen:
                continue

            self.validate_on[ct]['contexts']+=[ctx]
            self.validate_on[ct]['ENGINE']+=[eng[0]]
            self.validate_on[ct]['GEARBOX']+=[gb[0]]
            self.validate_on[ct]['SALES_VERSION']+=[sv[0]]
            self.validate_on[ct]['COL']+=[col[0]]
            self.validate_on[ct]['UPH']+=[uph[0]]
            self.validate_on[ct]['OPT']+=[opt[0]]     
    
    def fit(self,verbose=0,epochs=5):
        
        ctxs = []
        
        for ct in self.train_on.keys():
            ctxs += self.train_on[ct]['contexts']
            
        self.scaler.fit(ctxs)
        self.scaler_fitted = True
        
        t_contexts = []
        t_cts = []
        
        v_contexts = []
        v_cts = []
        
        def callbacks(name):
            dt_format = "%Y%m%d-%H%M%S"
            model_name = f"{name}_{datetime.now().strftime(dt_format)}"
            return [
                TensorBoard(log_dir='./logs/ksm/{}'.format(model_name)),
                ModelCheckpoint('./models/{}.mdl_wts.hdf5'.format(model_name),
                                save_best_only=True, monitor='val_loss', mode='min'),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose=1, mode='min'),
                EarlyStopping(restore_best_weights=True, patience=10)
            ]
        
        for ct in tqdm_notebook(self.train_on.keys()):

            gb_val=None
            eng_val=None
            sv_val = None
            ct_val = None
            col_val = None
            uph_val = None
            opt_val = None
            
            if (not self.validate_on is None) and (ct in self.validate_on):
                v_ctxs = np.array(self.scaler.transform(self.validate_on[ct]['contexts']))
                v_teng = np.array(self.validate_on[ct]['ENGINE'])
                v_tgbs = np.array(self.validate_on[ct]['GEARBOX'])
                v_tsv = np.array(self.validate_on[ct]['SALES_VERSION'])
                v_tcol = np.array(self.validate_on[ct]['COL'])
                v_tuph = np.array(self.validate_on[ct]['UPH'])
                v_topt = np.array(self.validate_on[ct]['OPT'])
                
                gb_val = ([v_ctxs,v_teng],v_tgbs)
                eng_val = (v_ctxs,v_teng)
                sv_val = (v_ctxs, v_tsv)
                col_val = (v_ctxs, v_tcol)
                uph_val = (v_ctxs, v_tuph)
                opt_val = (v_ctxs, v_topt)
                
                
                v_contexts += [ctx for ctx in v_ctxs]
                v_cts += [self.ct_encoder.transform([ct])[0] for _ in range(v_ctxs.shape[0])]
                

            encoders = self.label_encoders[ct]
            
            print('ct: ', ct)
            ctxs = np.array(self.scaler.transform(self.train_on[ct]['contexts']))
            teng = np.array(self.train_on[ct]['ENGINE'])
            tgbs = np.array(self.train_on[ct]['GEARBOX'])
            tsv = np.array(self.train_on[ct]['SALES_VERSION'])
            tcol = np.array(self.train_on[ct]['COL'])
            tuph = np.array(self.train_on[ct]['UPH'])
            topt = np.array(self.train_on[ct]['OPT'])
            
            #print('fitting svs',tsv)
            t_contexts += [ctx for ctx in ctxs]
            t_cts += [self.ct_encoder.transform([ct])[0] for _ in range(ctxs.shape[0])]
            #print(ctx_ct)
            #fitting gb
            print(f'fitting gb on {ct} with shapes ctx:{ctxs.shape}, teng:{teng.shape}, to: {tgbs.shape}')
            self.structures[ct]['GEARBOX'].fit([ctxs,teng],tgbs,validation_data=gb_val,verbose=verbose,epochs=epochs,callbacks=callbacks('gb'))
            #print('fitting engines',teng)

            print(f'fitting eng on {ct} with shapes ctx:{ctxs.shape}, to: {teng.shape}')

            self.structures[ct]['ENGINE'].fit(ctxs,teng,validation_data=eng_val,verbose=verbose,epochs=epochs,callbacks=callbacks('eng'))
            
            print(f'fitting sv on {ct} with shapes ctx:{ctxs.shape}, to: {tsv.shape}')
            
            self.structures[ct]['SALES_VERSION'].fit(ctxs,tsv,validation_data=sv_val,verbose=verbose,epochs=epochs,callbacks=callbacks('sv'))
            
            print(f'fitting col on {ct} with shapes ctx:{ctxs.shape}, to: {tcol.shape}')
            
            self.structures[ct]['COL'].fit(ctxs,tcol,validation_data=col_val,verbose=verbose,epochs=epochs,callbacks=callbacks('col'))
            
            print(f'fitting uph on {ct} with shapes ctx:{ctxs.shape}, to: {tuph.shape}')
            
            self.structures[ct]['UPH'].fit(ctxs,tuph,validation_data=uph_val,verbose=verbose,epochs=epochs,callbacks=callbacks('uph'))
                        
            print(f'fitting opt on {ct} with shapes ctx:{ctxs.shape}, to: {topt.shape}')
            
            self.structures[ct]['OPT'].fit(ctxs,topt,validation_data=opt_val,verbose=verbose,epochs=epochs,callbacks=callbacks('opt'))
            
            
        print(f'fitting cartypes to contexts')
        ct_val = None if len(v_contexts) == 0 else (np.array(v_contexts), np.array(v_cts))
        self.ct_predictor.fit(np.array(t_contexts),np.array(t_cts),validation_data=ct_val,verbose=verbose,epochs=epochs,callbacks=callbacks('ct'))
            
    def predict(self,context,threshold=0.4999):
        tctx = self.scaler.transform(context)
        cts = self.ct_encoder.inverse_transform(self.ct_predictor.predict(tctx))
        #print(cts)
        def transform(encs,structure,typ,inputs,multilabel=False):
            prediction = structure[typ].predict(inputs)
            if multilabel:
                oh = np.where(prediction[0] > threshold, 1, 0)
            else:
                oh = np.eye(prediction.shape[1])[prediction.argmax()]

            inv_= encs[typ].inverse_transform(oh.reshape(1,-1))

            return inv_

        pcts = []
        engs = []
        svs = []
        gbs = []
        cols = []
        uphs = []
        opts = []
        #print('here',tctx,cts)
        for ctx, ct in list(zip(tctx,cts)):
            structure = self.structures[ct]
            enc = self.label_encoders[ct]
            ctx_reshaped = ctx.reshape(1,-1)
            eng = transform(enc,structure,'ENGINE',ctx_reshaped)
            sv = transform(enc,structure,'SALES_VERSION',ctx_reshaped)
            gb = transform(enc,structure,'GEARBOX',[ctx_reshaped,enc['ENGINE'].transform(eng)])
            col = transform(enc,structure,'COL',ctx_reshaped)
            uph = transform(enc,structure,'UPH',ctx_reshaped)
            opt = transform(enc,structure,'OPT',ctx_reshaped,multilabel=True)
            
            pcts+=[self.cb.types_binarizer['CAR_TYPE'].transform([ct])[0]]
            engs+=[self.cb.types_binarizer['ENGINE'].transform(eng)[0]]
            svs+=[self.cb.types_binarizer['SALES_VERSION'].transform(sv)[0]]
            gbs+=[self.cb.types_binarizer['GEARBOX'].transform(gb)[0]]
            cols+=[self.cb.types_binarizer['COL'].transform(col)[0]]
            uphs+=[self.cb.types_binarizer['UPH'].transform(uph)[0]]
            opts+=[self.cb.types_binarizer['OPT'].transform(opt)[0]]

        return [pcts,engs,svs,gbs,cols,uphs,opts]
    
    def evaluate(self, df):
        self.stats = {
            'CAR_TYPE':0,
            'ENGINE':0,
            'GEARBOX':0,
            'SALES_VERSION':0,
            'COL':0,
            'UPH':0,
            'OPT':0,
            'TOTAL':0
        }

        predictions = self.predict(df[user_columns])
        idx = 0
        for ct,eng,sv,gb,col,uph,opts in tqdm_notebook(predictions):
            row = df.iloc[idx]
    
            if ct in row['TYPECODE_translated']:
                self.stats['CAR_TYPE'] += 1
            if eng in row['ENGINECODE_translated']:
                self.stats['ENGINE'] += 1
            if gb in row['GEARBOXCODE_translated']:
                self.stats['GEARBOX'] += 1
            if sv in row['SALESVERSIONCODE_translated']:
                self.stats['SALES_VERSION'] += 1
            if col in row['COLOUR_translated']:
                self.stats['COL'] += 1
            if uph in row['UPHOLSTERY_translated']:
                self.stats['UPH'] += 1

            for o in row['OPT_CODES_translated']:
                if o in opts:
                    self.stats['OPT']+=1

            idx += 1
        self.stats['TOTAL'] = idx
        return self.stats
    
        
bnn = BNN(model_dict,train_ct[user_columns].shape[1],cb=cb)


536_eng_model, computed output size: 6, specified size: 6
536_sv_model, computed output size: 5, specified size: 5
536_gearbox_model, computed output size: 5, specified size: 5
536_col_model, computed output size: 13, specified size: 13
536_uph_model, computed output size: 8, specified size: 8
536_opt_model, computed output size: 77, specified size: 77
234_eng_model, computed output size: 7, specified size: 7
234_sv_model, computed output size: 3, specified size: 3
234_gearbox_model, computed output size: 3, specified size: 3
234_col_model, computed output size: 13, specified size: 13
234_uph_model, computed output size: 26, specified size: 26
234_opt_model, computed output size: 96, specified size: 96
256_eng_model, computed output size: 4, specified size: 4
256_sv_model, computed output size: 8, specified size: 8
256_gearbox_model, computed output size: 1, specified size: 1
256_col_model, computed output size: 14, specified size: 14
256_uph_model, computed output size: 26, specified 

In [ ]:

for df in tqdm_notebook([train_ct ,train_sv,train_gb,train_eng,train_uph,train_col]):   
    train = df
    bnn.add_training_data(train[user_columns],train)
    
for df in tqdm_notebook([dev]):   
    devdata = df
    bnn.add_validation_data(devdata[user_columns],devdata)


In [ ]:
ksm.fit(verbose=2,epochs=20)

In [ ]:
evaluator = Evaluator(test,1,sample_size=10) #10 for now, just to show it
evaluator.enqueue(Pno34ModelResolver(ksm,cb,pno12_solver,pnotype=PNO12,use_solver=False), 'ksm_no_solver_pno12')
evaluator.enqueue(Pno34ModelResolver(ksm,cb,pno12_solver,pnotype=PNO34,use_solver=False), 'ksm_no_solver_pno34')
evaluator.enqueue(Pno34ModelResolver(ksm,cb,pno12_solver,pnotype=CIS,use_solver=False), 'ksm_no_solver_cis')


for t in np.arange(0.2,1.1,0.2):
    evaluator.enqueue(Pno34ModelResolver(ksm,cb,pno12_solver,threshold=t,pnotype=CIS,use_solver=True), f'ksm_w.solver_thres:{t}_CIS')

evaluator.enqueue(Pno34ModelResolver(ksm,cb,pno12_solver,pnotype=PNO12,use_solver=True), 'ksm_w.solver_pno12')
evaluator.enqueue(Pno34ModelResolver(ksm,cb,pno12_solver,pnotype=PNO34,use_solver=True), 'ksm_w.solver_pno34')
evaluator.enqueue(Pno34ModelResolver(ksm,cb,pno12_solver,pnotype=CIS,use_solver=True), 'ksm_w.solver_cis')


evaluator.run_queue()